In [1]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time
import pandas as pd

In [9]:
df = pd.read_csv(r"C:\Users\sungj\바탕 화면\project\DL\data\ratings.txt", sep = "\t")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [11]:
df = df.dropna()

In [12]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [13]:
df.shape

(199992, 3)

In [14]:
komoran = Komoran()

In [15]:
%%time
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
docs = df["document"].map(lambda x: komoran.nouns(x))

CPU times: total: 4min 6s
Wall time: 3min 29s


In [16]:
docs

0                                                       [때]
1         [디자인, 학생, 외국, 디자이너, 전통, 발전, 문화, 산업, 사실, 우리나라, ...
2                                    [폴리스, 스토리, 시리즈, 뉴, 최고]
3                                [연기, 것, 라고, 생각, 몰입, 영, 화지]
4                                        [안개, 밤하늘, 초승달, 영화]
                                ...                        
199995                                         [포켓, 몬스터, 짜]
199996                                                   []
199997                            [완전, 사이코, 영화, 마지막, 영화, 질]
199998                                          [라따뚜이, 스머프]
199999                       [포, 풍, 저그, 가나, 신다영, 차영, 차영, 차]
Name: document, Length: 199992, dtype: object

In [17]:
# word2vec 모델 학습
model = Word2Vec(sentences = docs, vector_size = 200, window = 4, min_count = 2, sg = 1)

- word2vec의 주요 하이퍼파라미터
    - sentences : Word2Vec 모델 학습에 필요한 문장 데이터
    - vector_size : 단어 임베딩 벡터의 차원(크기)
    - window : 주변 단어 윈도우의 크기
    - min_count : 단어 최소 빈도 수 제한(설정된 min_count 빈도 수 미만의 단어들은 학습하지 않음)
    - sg : 0(CBOW 모델), 1(skip-gram 모델)

In [18]:
# 모델 저장
model.save("./model/nsmc_w2v.model")

In [19]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count :", model.corpus_count)
print("corpus_total_words :", model.corpus_total_words)

corpus_count : 199992
corpus_total_words : 1076840


# word2vec 모델 활용

In [20]:
# 모델 로딩
model = Word2Vec.load("./model/nsmc_w2v.model")
print(model.corpus_total_words)

1076840


In [21]:
# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print("사랑 : ", model.wv["사랑"])

사랑 :  [ 8.26703086e-02 -3.38016748e-01  3.07669818e-01  4.48072016e-01
 -1.70633584e-01 -1.12956755e-01 -2.50643883e-02  7.63975307e-02
 -2.96293586e-01  4.56315935e-01 -2.33257994e-01 -8.06583315e-02
  3.45683604e-01 -2.55762279e-01 -1.70710683e-01  4.15638715e-01
 -1.72378927e-01 -5.21834016e-01 -2.63079017e-01 -3.40183496e-01
  4.57602471e-01  2.98333913e-01 -2.15921421e-02  1.01937279e-01
 -5.38865626e-01  1.28221199e-01  4.65089008e-02 -1.43279746e-01
 -6.32247422e-03 -6.48711249e-02  3.07931423e-01  2.05160454e-01
  1.90854222e-01  9.22632366e-02 -5.96402176e-02  1.55486479e-01
 -4.73330133e-02  8.95242300e-03 -1.80294201e-01 -1.52303157e-02
 -3.32703412e-01  1.74213782e-01  9.77888927e-02 -4.00540501e-01
  3.73107344e-01  3.97396266e-01 -2.43892282e-01 -1.31638497e-01
  1.58647001e-01  6.30471855e-02  2.89414167e-01 -1.49138272e-01
 -2.19394535e-01  2.23597199e-01 -1.82964936e-01 -1.95959568e-01
 -2.58924663e-01 -7.33260959e-02 -6.07650019e-02  2.45879330e-02
 -1.37612984e-01 -5

- 모델을 학습할 때 설정한 vector_size 하이퍼파라미터 만큼 단어 임베딩 벡터 차원 크기가 결정됨

In [22]:
# 단어 유사도 계산
print("일요일 = 월요일", model.wv.similarity(w1 = "일요일", w2 = "월요일"), sep = "\t")

일요일 = 월요일	0.8932427


In [23]:
print("대기업 = 삼성", model.wv.similarity(w1 = "대기업", w2 = "삼성"), sep = "\t")

대기업 = 삼성	0.8927722


In [24]:
print("일요일 = 삼성", model.wv.similarity(w1 = "일요일", w2 = "삼성"), sep = "\t")

일요일 = 삼성	0.6391073


- model.wv.most_similar() 함수 : 인자로 사용한 단어와 가장 유사한 단어를 리스트로 반환
    - 벡터 공간에서 가장 가까운 거리에 있는 단어들을 반환
    - topn : 반환되는 유사한 단어 수
- 유사도가 1에 가까울수록 두 단어는 동일한 의미이거나 문법적으로 관련이 있을 가능성이 높음

In [25]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("시리즈", topn = 5))

[('엑스맨', 0.8255574107170105), ('반지의 제왕', 0.8212547898292542), ('포터', 0.814788818359375), ('미이라', 0.7959827184677124), ('스타워즈', 0.7927154302597046)]


In [26]:
print(model.wv.most_similar("저그", topn = 5))

[('황신', 0.941754162311554), ('까르네', 0.9296473264694214), ('이승엽', 0.9217039942741394), ('진호', 0.9156176447868347), ('매운맛', 0.9133081436157227)]


In [28]:
print(model.wv.most_similar("왕", topn = 5))

[('끝판', 0.8225648999214172), ('죤', 0.7630272507667542), ('B1A4', 0.7404504418373108), ('혈', 0.7385879755020142), ('동', 0.7368876338005066)]


In [29]:
import gensim

In [30]:
sentences = gensim.models.word2vec.Text8Corpus("./data/text8.txt")

In [32]:
model = Word2Vec(sentences, vector_size = 200)

In [33]:
model.wv.most_similar(positive = ["woman", "king"], negative = ["man"])

[('queen', 0.6533477902412415),
 ('isabella', 0.5540904402732849),
 ('elizabeth', 0.5522816181182861),
 ('princess', 0.5517071485519409),
 ('daughter', 0.548427402973175),
 ('prince', 0.5477496385574341),
 ('empress', 0.5442290902137756),
 ('throne', 0.5383293032646179),
 ('monarch', 0.5293919444084167),
 ('son', 0.5289418697357178)]